In [1]:
import imageio
import torch
from PIL import Image
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
# dir(optim)

C:\Users\tyler\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\tyler\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
########################################################################
########################### Problem 1 Part 1 ###########################
########################################################################

In [2]:
import pandas as pd
housing = pd.DataFrame(pd.read_csv("Housing.csv"))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
m = len(housing)
housing.shape

(545, 13)

In [4]:
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking','price'] 
Newtrain = housing[num_vars] 
Newtrain.head()

area = torch.tensor(Newtrain['area'])
bedrooms = torch.tensor(Newtrain['bedrooms'])
bathrooms = torch.tensor(Newtrain['bathrooms'])
stories = torch.tensor(Newtrain['stories'])
parking = torch.tensor(Newtrain['parking'])
price = torch.tensor(Newtrain['price'])

In [5]:
val_frac = 0.8
num_labels = np.random.rand(len(housing)) <= val_frac

area_val = area[~num_labels]
area_val = area_val/(max(area_val))
bedrooms_val = bedrooms[~num_labels]
bedrooms_val = bedrooms_val/(max(bedrooms_val))
bathrooms_val = bathrooms[~num_labels]
bathrooms_val = bathrooms_val/(max(bathrooms_val))
stories_val = stories[~num_labels]
stories_val = stories_val/(max(stories_val))
parking_val = parking[~num_labels]
parking_val = parking_val/(max(parking_val))
price_val = price[~num_labels]
price_val = price_val/(max(price_val))

area_train = area[num_labels]
area_train = area_train/(max(area_train))
bedrooms_train = bedrooms[num_labels]
bedrooms_train = bedrooms_train/(max(bedrooms_train))
bathrooms_train = bathrooms[num_labels]
bathrooms_train = bathrooms_train/(max(bathrooms_train))
stories_train = stories[num_labels]
stories_train = stories_train/(max(stories_train))
parking_train = parking[num_labels]
parking_train = parking_train/(max(parking_train))
price_train = price[num_labels]
price_train = price_train/(max(price_train))

train_list = [area_train, bedrooms_train, bathrooms_train, stories_train, parking_train]
t_u_train = torch.stack(train_list)
t_u_train = np.transpose(t_u_train)
val_list = [area_val, bedrooms_val, bathrooms_val, stories_val, parking_val]
t_u_val = torch.stack(val_list)
t_u_val = np.transpose(t_u_val)
print(t_u_val.shape)
print(t_u_train.shape)

torch.Size([108, 5])
torch.Size([437, 5])


In [6]:
def training_loop(n_epochs, optimizer, model, loss_fn, 
                  t_u_train, t_u_val,
                  price_train, price_val):
    for epoch in range(1, n_epochs+1):
        t_p_train = model(t_u_train.float())
        loss_train = loss_fn(t_p_train, price_train)
        
        t_p_val = model(t_u_val.float())
        loss_val = loss_fn(t_p_val, price_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"f" Validation loss {loss_val.item():.4f}")


In [7]:
seq_model = nn.Sequential(
            nn.Linear(5,8),
            nn.Tanh(),
            nn.Linear(8,1))
seq_model

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [8]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)

In [9]:
training_loop(
n_epochs = 200,
optimizer = optimizer,
model = seq_model,
loss_fn = nn.MSELoss(),
t_u_train = t_u_train, t_u_val = t_u_val,
price_train = price_train, price_val = price_val)

Epoch 1, Training loss 0.0649, Validation loss 0.0537
Epoch 2, Training loss 0.0617, Validation loss 0.0512
Epoch 3, Training loss 0.0588, Validation loss 0.0489
Epoch 4, Training loss 0.0561, Validation loss 0.0468
Epoch 5, Training loss 0.0536, Validation loss 0.0449
Epoch 6, Training loss 0.0513, Validation loss 0.0431
Epoch 7, Training loss 0.0492, Validation loss 0.0415
Epoch 8, Training loss 0.0472, Validation loss 0.0401
Epoch 9, Training loss 0.0454, Validation loss 0.0387
Epoch 10, Training loss 0.0437, Validation loss 0.0375
Epoch 11, Training loss 0.0422, Validation loss 0.0364
Epoch 12, Training loss 0.0407, Validation loss 0.0354
Epoch 13, Training loss 0.0394, Validation loss 0.0345
Epoch 14, Training loss 0.0381, Validation loss 0.0337
Epoch 15, Training loss 0.0370, Validation loss 0.0329
Epoch 16, Training loss 0.0359, Validation loss 0.0322
Epoch 17, Training loss 0.0349, Validation loss 0.0316
Epoch 18, Training loss 0.0340, Validation loss 0.0311
Epoch 19, Training 

C:\Users\tyler\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([437])) that is different to the input size (torch.Size([437, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\tyler\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([108])) that is different to the input size (torch.Size([108, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
########################################################################
########################### Problem 2 Part 1 ###########################
########################################################################

In [10]:
seq_model2 = nn.Sequential(
            nn.Linear(5,10),
            nn.Tanh(),
            nn.Linear(10,8),
            nn.Tanh(),
            nn.Linear(8,4),
            nn.Tanh(),
            nn.Linear(4,1))
seq_model2

Sequential(
  (0): Linear(in_features=5, out_features=10, bias=True)
  (1): Tanh()
  (2): Linear(in_features=10, out_features=8, bias=True)
  (3): Tanh()
  (4): Linear(in_features=8, out_features=4, bias=True)
  (5): Tanh()
  (6): Linear(in_features=4, out_features=1, bias=True)
)

In [11]:
optimizer2 = optim.SGD(seq_model2.parameters(), lr=1e-2)

In [12]:
training_loop(
n_epochs = 200,
optimizer = optimizer2,
model = seq_model2,
loss_fn = nn.MSELoss(),
t_u_train = t_u_train, t_u_val = t_u_val,
price_train = price_train, price_val = price_val)

Epoch 1, Training loss 0.7817, Validation loss 0.7646
Epoch 2, Training loss 0.7231, Validation loss 0.7068
Epoch 3, Training loss 0.6694, Validation loss 0.6539
Epoch 4, Training loss 0.6200, Validation loss 0.6052
Epoch 5, Training loss 0.5745, Validation loss 0.5606
Epoch 6, Training loss 0.5327, Validation loss 0.5194
Epoch 7, Training loss 0.4941, Validation loss 0.4816
Epoch 8, Training loss 0.4586, Validation loss 0.4467
Epoch 9, Training loss 0.4257, Validation loss 0.4144
Epoch 10, Training loss 0.3954, Validation loss 0.3847
Epoch 11, Training loss 0.3673, Validation loss 0.3572
Epoch 12, Training loss 0.3414, Validation loss 0.3319
Epoch 13, Training loss 0.3174, Validation loss 0.3084
Epoch 14, Training loss 0.2952, Validation loss 0.2867
Epoch 15, Training loss 0.2746, Validation loss 0.2667
Epoch 16, Training loss 0.2556, Validation loss 0.2481
Epoch 17, Training loss 0.2379, Validation loss 0.2310
Epoch 18, Training loss 0.2216, Validation loss 0.2151
Epoch 19, Training 

In [ ]:
########################################################################
########################### Problem 2 Part 1 ###########################
########################################################################

In [13]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))

Normalize(mean=(0.4915, 0.4823, 0.4468), std=(0.247, 0.2435, 0.2616))

In [16]:
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915,0.4823,0.4468),
                          (0.2470,0.2435,0.2616))
]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915,0.4823,0.4468),
                          (0.2470,0.2435,0.2616))
]))

Files already downloaded and verified
Files already downloaded and verified


In [17]:
model = nn.Sequential(
              nn.Linear(3072,512),
              nn.Tanh(),
              nn.Linear(512,10),
              nn.LogSoftmax(dim=1))
model.to(device)
loss_fn = nn.NLLLoss()

In [18]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)


In [19]:
learning_rate = 1e-3

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 300

for epoch in range(n_epochs):
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)
        batch_size = img.shape[0]
        out = model(img.view(batch_size, -1))
        loss = loss_fn(out, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 1.791856
Epoch: 1, Loss: 1.621083
Epoch: 2, Loss: 1.808523
Epoch: 3, Loss: 1.811121
Epoch: 4, Loss: 1.843995
Epoch: 5, Loss: 1.564411
Epoch: 6, Loss: 1.902052
Epoch: 7, Loss: 2.233577
Epoch: 8, Loss: 1.736857
Epoch: 9, Loss: 1.884543
Epoch: 10, Loss: 1.898587
Epoch: 11, Loss: 1.577592
Epoch: 12, Loss: 1.464556
Epoch: 13, Loss: 1.904310
Epoch: 14, Loss: 1.744929
Epoch: 15, Loss: 1.742007
Epoch: 16, Loss: 2.032099
Epoch: 17, Loss: 1.736929
Epoch: 18, Loss: 1.605575
Epoch: 19, Loss: 1.982994
Epoch: 20, Loss: 1.438289
Epoch: 21, Loss: 1.433143
Epoch: 22, Loss: 1.505153
Epoch: 23, Loss: 1.514912
Epoch: 24, Loss: 1.879959
Epoch: 25, Loss: 1.976578
Epoch: 26, Loss: 1.566835
Epoch: 27, Loss: 1.598035
Epoch: 28, Loss: 1.746503
Epoch: 29, Loss: 1.342918
Epoch: 30, Loss: 1.219455
Epoch: 31, Loss: 1.901271
Epoch: 32, Loss: 1.403945
Epoch: 33, Loss: 1.728805
Epoch: 34, Loss: 1.388762
Epoch: 35, Loss: 1.762833
Epoch: 36, Loss: 1.682974
Epoch: 37, Loss: 2.258845
Epoch: 38, Loss: 1.304

In [34]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
print("Accuracy: %f", (correct/total))

Accuracy: %f 0.4951


In [ ]:
########################################################################
########################### Problem 2 Part 2 ###########################
########################################################################

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [51]:
model2 = nn.Sequential(
              nn.Linear(3072,512),
              nn.Tanh(),
              nn.Linear(512,256),
              nn.Tanh(),
              nn.Linear(256,128),
              nn.Tanh(),
              nn.Linear(128,10),
              nn.LogSoftmax(dim=1))
model2.to(device)
loss_fn = nn.NLLLoss()

In [52]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

In [53]:
learning_rate = 1e-3

optimizer = optim.SGD(model2.parameters(), lr=learning_rate)

n_epochs = 300

for epoch in range(n_epochs):
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)
        batch_size = img.shape[0]
        out = model2(img.view(batch_size, -1))
        loss = loss_fn(out, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.144830
Epoch: 1, Loss: 2.029987
Epoch: 2, Loss: 1.782338
Epoch: 3, Loss: 1.955189
Epoch: 4, Loss: 1.853665
Epoch: 5, Loss: 1.811280
Epoch: 6, Loss: 1.902127
Epoch: 7, Loss: 2.071814
Epoch: 8, Loss: 1.822491
Epoch: 9, Loss: 1.687687
Epoch: 10, Loss: 1.771420
Epoch: 11, Loss: 1.874066
Epoch: 12, Loss: 1.887277
Epoch: 13, Loss: 1.960525
Epoch: 14, Loss: 1.775205
Epoch: 15, Loss: 1.542766
Epoch: 16, Loss: 1.366641
Epoch: 17, Loss: 1.388609
Epoch: 18, Loss: 1.415257
Epoch: 19, Loss: 1.939723
Epoch: 20, Loss: 1.847245
Epoch: 21, Loss: 1.957929
Epoch: 22, Loss: 1.537865
Epoch: 23, Loss: 1.611171
Epoch: 24, Loss: 1.703014
Epoch: 25, Loss: 1.554873
Epoch: 26, Loss: 1.619266
Epoch: 27, Loss: 1.433541
Epoch: 28, Loss: 1.371979
Epoch: 29, Loss: 1.746187
Epoch: 30, Loss: 1.357478
Epoch: 31, Loss: 1.963365
Epoch: 32, Loss: 1.869833
Epoch: 33, Loss: 1.577680
Epoch: 34, Loss: 1.532642
Epoch: 35, Loss: 1.531450
Epoch: 36, Loss: 1.405720
Epoch: 37, Loss: 1.436036
Epoch: 38, Loss: 1.524

In [54]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = model2(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
print("Accuracy: %f", (correct/total))

Accuracy: %f 0.4383
